In [2]:
import duckdb
cn = duckdb.connect()

In [29]:
data_sql = """
    select t.row_id, uuid() as txn_key, current_date as rpt_dt
        ,round(random() * 100,2) as some_val
    from generate_series(1,{rows}) t(row_id)
"""

# to sample
cn.sql(data_sql.format(rows=5)).show()

parquet_sql = f"COPY ({data_sql.format(rows=50000000)}) TO '~/test_dummy_data/duckdb/data1.parquet' (FORMAT PARQUET)"

cn.sql(parquet_sql)

┌────────┬──────────────────────────────────────┬────────────┬──────────┐
│ row_id │               txn_key                │   rpt_dt   │ some_val │
│ int64  │                 uuid                 │    date    │  double  │
├────────┼──────────────────────────────────────┼────────────┼──────────┤
│      1 │ 487012ae-5f21-4c08-a58d-1a336f0aa082 │ 2024-06-03 │    86.98 │
│      2 │ 03c0db35-3aa6-4b88-961c-fa60a775b766 │ 2024-06-03 │    24.76 │
│      3 │ 9dcca4b4-2fe2-4bcc-bda7-e384cb76c6bd │ 2024-06-03 │    92.82 │
│      4 │ f1346cb5-6cfd-493a-8925-3038adf9e582 │ 2024-06-03 │    56.97 │
│      5 │ ed9e7129-7159-433e-881d-c590a63fa75b │ 2024-06-03 │    19.96 │
└────────┴──────────────────────────────────────┴────────────┴──────────┘



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/04 14:50:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [17]:
import uuid, random, os
from pyspark.sql.functions import current_date, udf
from pyspark.sql.types import StringType, IntegerType

def crt_uuid():
    return str(uuid.uuid4())

def crt_int():
    return random.randint(0, 100)

s_crt_uuid = udf(crt_uuid, StringType())
s_crt_int = udf(crt_int, IntegerType())

home_dir = os.path.expanduser("~")
print(home_dir)

/Users/matthewmartin


In [20]:
df = spark.range(1,5+1) \
    .withColumn("uuid", s_crt_uuid()) \
    .withColumn("rpt_dt", current_date()) \
    .withColumn("rand_val", s_crt_int()) \
    .toDF('row_id', 'uuid', 'rpt_dt', 'rand_val')
#df.show()

f_path = os.path.join(home_dir, "test_dummy_data/spark/")
print(f_path)

df.write.mode('overwrite').parquet(f_path)


/Users/matthewmartin/test_dummy_data/spark/
